# Lawer ChatBot for Legal Help
## by Abhisek Sarkar

In [25]:
# Importing necessery packages
import os
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq

In [26]:
# === Configurations ===
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
FAISS_INDEX_PATH = '/home/abhisek/Project/AI_Lawer_ChatBot/JSONS_embedding/faiss_index'
FAISS_META_PATH = '/home/abhisek/Project/AI_Lawer_ChatBot/JSONS_embedding/faiss_index_meta.pkl'
GROQ_API_KEY = "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"

In [27]:
# === Groq Client ===
client = Groq(api_key=GROQ_API_KEY)

In [28]:
# === Load embedding model ===
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [29]:
# === Load FAISS index and metadata ===
def load_faiss_index():
    index = faiss.read_index(FAISS_INDEX_PATH)
    with open(FAISS_META_PATH, 'rb') as f:
        metadata = pickle.load(f)
    return index, metadata


In [30]:
index, metadata_list = load_faiss_index()

In [31]:
# === Embedding function ===
def embed_text(text):
    return embedding_model.encode([text], normalize_embeddings=True)

In [32]:
# === Search function ===
def search_faiss(query, top_k=5):
    query_embedding = embed_text(query)
    distances, indices = index.search(query_embedding, top_k)
    return [metadata_list[i] for i in indices[0] if i < len(metadata_list)]

In [33]:
# === Prompt construction ===
def build_prompt(contexts, user_query):
    context_text = "\n\n".join(
        f"Section {ctx.get('section', 'N/A')}: {ctx.get('title', '')}\n{ctx.get('description', '')}"
        for ctx in contexts
    )
    
    return f"""You are a legal expert AI specialized in Indian Law. Using *only* the legal sections retrieved below, answer the user's question in a clear, authoritative, and concise manner.

Rules:
- Cite the relevant law sections exactly.
- Do NOT add imagined legal advice.
- Be to-the-point, like a real Indian legal expert.
- If no law section applies, say: "No relevant legal section found for your query."

Retrieved Law Sections:
{context_text}

User Query:
{user_query}

Now answer the query based strictly on the law sections above.
"""

In [34]:
# === Groq LLM call ===
def call_groq(prompt, model="deepseek-r1-distill-llama-70b"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a legal expert on Indian Law."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=1024
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [35]:
# === Interactive CLI ===
def main():
    print("⚖️ Indian Legal Expert RAG Bot")
    print("Type your legal query or 'exit' to quit.\n")

    while True:
        user_query = input("👤 You: ").strip()
        print("Your query is:")
        print(user_query)
        if user_query.lower() == "exit":
            print("👋 Exiting. Stay legally informed!")
            break

        print("\n🔎 Searching for relevant sections...")
        matched_sections = search_faiss(user_query)

        if not matched_sections:
            print("No relevant law sections found.")
            continue

        print("\n📄 Top Legal Matches:")
        for i, sec in enumerate(matched_sections, 1):
            print(f"{i}. Section {sec.get('section', 'N/A')} - {sec.get('title', '')}")

        full_prompt = build_prompt(matched_sections, user_query)

        print("\n🧠 Generating legal response...\n")
        response = call_groq(full_prompt)

        print("⚖️ Expert Legal Opinion:")
        print(response)
        print("\n" + "="*60 + "\n")

In [36]:
if __name__ == "__main__":
    main()


⚖️ Indian Legal Expert RAG Bot
Type your legal query or 'exit' to quit.

Your query is:
Can a Christian woman file for divorce on grounds of cruelty in India?

🔎 Searching for relevant sections...

📄 Top Legal Matches:
1. Section 10 - Grounds for dissolution of marriage
2. Section 2 - Extent of Act
3. Section 498A - Husband or relative of husband of a woman subjecting her to cruelty
4. Section 22 - Bar to decree for divorce a mensa et toro; but judicial separation obtainable by husband or wife
5. Section 25 - Separated wife deemed spinster for purposes of contract and suing

🧠 Generating legal response...

⚖️ Expert Legal Opinion:
<think>
Okay, so I need to figure out if a Christian woman can file for divorce on the grounds of cruelty in India. Let me start by looking at the law sections provided.

First, Section 10 lists the grounds for dissolution of marriage. It includes cruelty as a ground in clause (x). That means cruelty is a valid reason for divorce under this section.

Next, I 